In [0]:
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
labels = ['airplane',
          'airport',
          'baseball diamond',
          'basketball court',
          'beach',
          'bridge',
          'chaparral',
          'church',
          'circular farmland',
          'cloud',
          'commercial area',
          'dense residential',
          'desert',
          'forest',
          'freeway']

In [0]:
preprocess = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225]),
])

trainset = datasets.ImageFolder('/content/drive/My Drive/ComputerVision/data/', transform=preprocess)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = datasets.ImageFolder('/content/drive/My Drive/ComputerVision/data/', transform=preprocess)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=True)

In [0]:
model = models.vgg16(pretrained=True)
for param in model.parameters():
  param.requires_grad = False

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/checkpoints/vgg16-397923af.pth


In [0]:
classifier_input = model.classifier.in_features
num_labels = 15

classifier = nn.Sequential(nn.Linear(classifier_input, 1024),
                           nn.ReLU(),
                           nn.Linear(1024, 512),
                           nn.ReLU(),
                           nn.Linear(512, num_labels),
                           nn.LogSoftmax(dim=1))

model.classifier = classifier

In [0]:
criterion = nn.NLLLoss()

optimizer = optim.Adam(model.classifier.parameters())

epochs = 10
for epoch in range(epochs):

  train_loss = 0
  test_loss = 0
  accuracy = 0

  model.train()
  counter = 0
  for inputs, labels in trainloader:
    optimizer.zero_grad()
    output = model.forward(inputs)
    loss = criterion(output, labels)
    loss.requires_grad = True
    loss.backward()
    optimizer.step()
    train_loss += float(loss.item())*inputs.size()
    counter += 1
    print(counter, "/", len(trainloader))
  
  model.eval()
  counter = 0

  with torch.no_grad():
    for inputs, labels in testloader:
      output = model.forward(inputs)
      loss = criterion(output, labels)
      train_loss += float(loss.item())*inputs.size(0)
      output = torch.exp(output)
      top_p, top_class = output.topk(1, dim=1)
      equals = top_class == labels.view(*top_class.shape)
      accuracy += torch.mean(equals.type(torch.FloatTensor)).item()

      counter += 1
      print(counter, "/", len(testloader))
      
    train_loss = train_loss/len(trainloader.dataset)
    valid_loss = test_loss/len(testloader.dataset)

    print('Accuracy: ', accuracy/len(val_loader))
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss:{:.6f}'.format(epoch, train_loss, valid_loss))